In [1]:
import fair
import gym
from stable_baselines3 import PPO, DDPG, A2C

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random
import math
from functools import partial
from scipy.stats import gamma

import json
import os
import sys
import time

import fair
from fair.RCPs import rcp26, rcp45, rcp60, rcp85
from fair.SSPs import ssp370, ssp245, ssp585

/Users/samart/opt/anaconda3/envs/MLforEE/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'fair.RCPs'

In [ ]:
#Economic dimension
Y = np.zeros(356+300) #global GDP
Y_cost = np.zeros(356+300) #cost of climate change
S = np.zeros(356+300) #renewable knowledge stock
S[255] = 5e11 #GJ
Y[255] = 9e13 #USD/a
Y_cost[255] = 100*1e9 #USD/a
S[256] = 5e11 #GJ
Y[256] = 9e13 #USD/a
Y_cost[256] = 100*1e9 #USD/a
beta = 0.03 # 1/yr
epsilon = 147. # USD/GJ
rho = 2. # 1
sigma = 4.e12 # GJ
tau_S = 65. # yr


# Labels for plots
VARS = [
    "Temperature anomaly (ºC)",
    "CO2 Emissions (GtC)",
    "CO2 Concentration (ppm)",
    "Radiative forcing (W m^-2)",
    "Reward "
]

MULTIGAS_VARS = [
    "Temperature anomaly (ºC)",
    "CO2 Emissions (GtC)",
    "CO2 Concentration (ppm)",
    "CO2 forcing (W m^-2)",
    "CH4 forcing (W m^-2)",
    "N2O forcing (W m^-2)",
    "All other well-mixed GHGs forcing (W m^-2)",
    "Tropospheric O3 forcing (W m^-2)",
    "Stratospheric O3 forcing (W m^-2)",
    "Stratospheric water vapour from CH4 oxidation forcing (W m^-2)",
    "Contrails forcing (W m^-2)",
    "Aerosols forcing (W m^-2)",
    "Black carbon on snow forcing (W m^-2)",
    "Land use change forcing (W m^-2)",
    "Volcanic forcing (W m^-2)",
    "Solar forcing (W m^-2)",
    "Reward ",
]

#### Reward function options ####
def simple_reward(state, cur_temp, t, cur_emit, cur_conc, cur_GDP, cur_fease):
# positive reward for temp decrease
# negative cliff if warming exceeds 2º
    if cur_temp > 2:
        return -100
    return (state[0] - cur_temp)

def temp_reward(state, cur_temp, t, cur_emit, cur_conc, cur_GDP, cur_fease):
# positive reward for temp under 1.5 goal
    if cur_temp > 2:
        return -100
    return 1.5 - cur_temp

def conc_reward(state, cur_temp, t, cur_emit, cur_conc, cur_GDP, cur_fease):
    # positive reward for decreased concentration
    if cur_temp > 2:
        return -100
    return state[2] - cur_conc

def carbon_cost_reward(state, cur_temp, t, cur_emit, cur_conc, cur_GDP, cur_fease):
# impose a cost for each GtC emitted
    if cur_temp > 2:
        return -100
    return -cur_emit

def carbon_cost_GDP_reward(state, cur_temp, t, cur_emit, cur_conc, cur_GDP, cur_fease):
    reward = 0
    if cur_temp > 2:
        reward += -1000
    if cur_fease < 0:
        reward += -1000
    if cur_fease > 0:
        reward += -10*cur_fease
    reward += -cur_PIB/1e11
    return reward

def temp_emit_reward(state, cur_temp, t, cur_emit, cur_conc, cur_GDP, cur_fease):
    # positive reward for keeping the temp under 1.5
    # negative reward for amount of emissions reduction
    # positive cliff for success at the end of the trial
    # w could indicate cost
    if cur_temp > 2:
        return -100
    if t==79 and temp <=1.5:
        return 100
    temp = 10*(state[0] - cur_temp)
    emit = state[1] - cur_emit
    if cur_emit < state[1]:
        return temp - emit
    return temp


def temp_emit_diff_reward(state, cur_temp, t, cur_emit, cur_conc, cur_GDP, cur_fease):
    # positive reward for keeping the temp under 1.5
    # negative reward for amount of emissions reduction
    # (reduction compared to projected amount for that year)
    # positive cliff for success at the end of the trial
    # w could indicate cost of emissions
    if cur_temp > 2:
        return -100
    if t==79 and temp <=1.5:
        return 100
    curval = t*0.6 + 36
    temp = 10*(state[0] - cur_temp)
    emit = curval - cur_emit
    if cur_emit < curval:
        return temp - emit
    return temp

In [21]:
gamma.rvs(0.2, size=5, random_state=14)

array([0.03585719, 0.00285074, 0.03556975, 0.04557736, 0.34139777])

In [22]:
"""
Create an Environment class and extend following functions:

1. Constructor- initialize continuous action space and observation space, select reward function, 
    define time advancement function from FaIR. 

2. update_state takes in C, F, T which corresponds to emission data and changes params of self.state and self.t

3. reset() sets all emissions to baseline from provided SSP scenario

4. step(action) takes in action and computes forward emissions data from environment. updates state with what is computed and returns reward

5. render() plots all the relevant climate info. 


""" 



class Simulator(gym.Env):
    def __init__(self, action_space=36, reward_mode="temp_emit_reward", climate_params={},
                 forcing=False, multigas=True, scenario='ssp245', verbose=1):
        
        # action space for the environment,
        # the amount to increase or decrease emissions by
        self.action_space = gym.spaces.Box(
            np.array([-action_space]).astype(np.float32),
            np.array([+action_space]).astype(np.float32),
        )
        # state space, [temperature, carbon emissions, carbon concentration, radiative forcing]
        if not multigas:
            self.observation_space = gym.spaces.Box(
                np.array([-100, -100, 0, -100]).astype(np.float32),
                np.array([100, 100, 5000, 100]).astype(np.float32),
            )
        else:
            self.observation_space = gym.spaces.Box( # both length 16
                np.array([-100, -100, 0, -50, -50, -50, -50, -50, -50, -50, -50,
                -50, -50, -50, -50, -50]).astype(np.float32),
                
                np.array([100, 100, 5000, 50, 50, 50, 50, 50, 50, 50, 50, 50,
                50, 50, 50, 50]).astype(np.float32),
            )
        
        # specify the reward function to use
        
        self.reward_func = eval(reward_mode)

        # setup climate parameters for FaIR model
        
        
        if forcing:
            climate_params['F_solar'] = 0.1 * np.sin(2 * np.pi * np.arange(736) / 11.5)
            climate_params['F_volcanic'] = -gamma.rvs(0.2, size=736, random_state=14) # samples from gamma probability distribution
            
        self.forward_func = partial(fair.forward.fair_scm, **climate_params)
        
        self.multigas = multigas
        self.scenario = scenario
        self.verbose = verbose
        
        # set the initial state
        self.reset()
        
        
    def update_state(self, C, F, T):
        if self.multigas:
            concentration = C[:,0][256+self.t]
            forcing = np.sum(F,axis=1)[256+self.t]
            emissions = self.emissions[:,1][256+self.t]
        else:
            concentration = C[256+self.t]
            forcing = F[256+self.t]
            emissions = self.emissions[256+self.t]
        self.state = [T[256+self.t], emissions, concentration] + [forcing]
        self.t += 1
        
        
    def reset(self):
        base_emissions = eval(self.scenario).Emissions.emissions
        if not self.multigas:
            base_emissions = np.array([x[1]+x[2] for x in base_emissions])
        
        # 80 year time horizon, meet goals by 2100
        self.emissions = base_emissions
        # 2021 estimate of GtC of carbon emissions
        # 2021 is the 257th year in the ssp scenario
    
        self.t = 0
        # initial state
        C, F, T = self.forward_func( #forward func computes from FaIR climate simulator, returns world
            emissions=self.emissions,
            useMultigas=True,
        )
        
        self.update_state(C,F,T)
        return(self.state)
    
    
    def step(self, action):
        done = False
        # change emissions by the action amount
        
        if not self.multigas:
            # change time step to be previous time step + delta E from action (negative). Floor is zero
            self.emissions[256+self.t] = max(self.emissions[256+self.t-1] + action[0], 0)
        else:
            self.emissions[:,1][256+self.t] = max(self.emissions[:,1][256+self.t-1] + action[0]*.9, 0)
            self.emissions[:,2][256+self.t] = max(self.emissions[:,2][256+self.t-1] + action[0]*.1, 0)
        # run FaIR simulator
        C, F, T = self.forward_func(
            emissions=self.emissions,
            useMultigas= True,
        )
        
        #Implementation of S, Y and Y_cost
        
        gamma = 1 / ( 1+(S[256+self.t-1]/sigma)**rho )
        Y[256+self.t] = Y[256+self.t-1] + beta*Y[256+self.t-1]
        Y_cost[256+self.t] = (10/5*T[256+self.t]-2)/100*Y[256+self.t]
        S[256+self.t] = S[256+self.t-1] + ( (1-gamma)*Y[256+self.t-1]/epsilon - S[256+self.t-1]/tau_S )
        # fail if temperature error
        if math.isnan(T[256+self.t]):
            done = True
            
            
        # determine economic and other parameters to feed into loss function
        if self.multigas:
            cur_emit = self.emissions[:,1][256+self.t] + self.emissions[:,2][256+self.t]
            cur_conc = C[:,0][256+self.t]
            cur_fease = ssp_370[self.t-1] - cur_emit
        else:
            cur_emit = self.emissions[256+self.t]
            cur_fease = ssp_370[self.t-1] - cur_emit
            cur_conc = C[256+self.t]
        cur_GDP = Y_cost[256+self.t]
        
        #compute the reward
        reward = self.reward_func(self.state, T[256+self.t], self.t, cur_emit, cur_conc, cur_PIB, cur_fease)
        
        # update the state and info
        self.update_state(C, F, T)
        # end the trial once 2100 is reached
        if self.t == 79 or self.state[0] > 4 or self.state[0] < 0: # only runs to 2100
            done = True
        return self.state, reward, done, {}
    
    
    def render(self, mode="human"):
        # print the state
        print(f'Temperature anomaly: {self.state[0]}ºC')
        print(f'CO2 emissions: {self.state[1]} GtC')
        print(f'CO2 concentration: {self.state[2]} ppm')
        print(f'Radiative forcing: {self.state[3:]}')
        




In [47]:
args = {
    'name' : "test_runs", #the path in which the run will be saved
    "output_path": "outputs",
    "stdout": False,
    'action_space' : 36,
    "seed": 158,
    'reward_mode' : "temp_emit_diff_reward",
    'forcing' : True,
    'multigas' : False,
    'scenario' : "ssp245",
    'algorithm' : "a2c",
    'learning_rate' : 2.1e-05,
    'num_steps' : 100, 
    'gamma' : 0.9, 
    'device' : 'cpu',
    'timestep' : 200,
    "n_steps" : 5,
    
}


In [48]:
env = Simulator(args['action_space'], reward_mode=args['reward_mode'], climate_params=climate, 
        forcing=False, multigas=False,scenario='ssp245', verbose=1)

env.reset()

IndexError: tuple index out of range

In [35]:
save_path = os.path.join()

In [29]:
emissions = np.zeros(250) # CO2 emissions in every year, nx1
emissions[125:] = 10.0
forcing = 0.5 * np.sin(2 * np.pi *  np.arange(250) / 14)
C, F, T = env.forward_func(emissions=emissions, other_rf=forcing, useMultigas=False)
C[-10:]

array([536.49627845, 538.84369225, 541.22506941, 543.62995889,
       546.04442051, 548.45353445, 550.84404035, 553.20679299,
       555.5386549 , 557.84338074])

In [46]:
climate_env = Simulator(
    action_space = args["action_space"],
    reward_mode = args['reward_mode'],
    forcing = args['forcing'],
    multigas = args['multigas'],
    scenario = args['scenario']
)

model_builder = eval(args['algorithm'].upper())

model = model_builder(
    policy="MlpPolicy",
    env=climate_env,
    learning_rate = args['learning_rate'],
    n_steps = args['num_steps'],
    gamma= args['gamma'],
    verbose=1,
    tensorboard_log=os.path.join(save_path, 'logs'),
)

model.learn(total_timesteps=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: could not broadcast input array from shape (4,) into shape (16,)

In [43]:
model.learn(
    total_timesteps=args['timestep'],
    eval_freq=20,
    log_interval=20,
    eval_log_path=os.path.join(save_path, 'evals')
)
model.save(f'{save_path}/model_state_dict.pt')

/var/folders/lc/mjy2_vc10zd7k8yny4sjghbr0000gn/T/ipykernel_12100/118988271.py:1: DeprecationWarning: Parameters `eval_env` and `eval_freq` are deprecated and will be removed in the future. Please use `EvalCallback` or a custom Callback instead.
  model.learn(


AttributeError: 'Simulator' object has no attribute 'num_envs'

In [27]:
#### Training code ####
# Output useful plots
def make_plots(vals, args, save_path):
    plots = MULTIGAS_VARS if args.multigas else VARS
    
    for i in range(len(plots)):
        name = plots[i][:plots[i].find('(')].strip()
        xs = [2021+x for x in range(len(vals))]
        ys = [x[i] for x in vals]
        plt.plot(xs, ys)
        plt.ylabel(plots[i])
        plt.xlabel('Year')
        plt.savefig(os.path.join(save_path, 'plots', name))
        plt.clf()
    excel = pd.DataFrame({
        "Years" : [2021+x for x in range(len(vals))],
        "Temperature anomaly" : [x[0] for x in vals],
        "CO2 Emissions" : [x[1] for x in vals],
        "CO2 Concentration" : [x[2] for x in vals],
        "Radiative forcing" : [x[3] for x in vals],
        "Reward" : [x[4] for x in vals]
    })
    
    with open(os.path.join(save_path, 'excel.xls'), 'w') as f:
        excel.to_excel (save_path + 'excel.xls')
        #json.dump(excel, f, indent=None, separators=None)

Learns  from stable_baselines3 import PPO, DDPG, A2C models which have model_builder and train methods. 

In [52]:
def run_model(args, save_path):
    # Create the environment
    env = Simulator(
        action_space = args["action_space"],
        reward_mode = args['reward_mode'],
        forcing = args['forcing'],
        multigas = args['multigas'],
        scenario = args['scenario']
    )
    
    # Train the algorithm
    model_builder = eval(args['algorithm'].upper())

    
    model = model_builder(
        policy="MlpPolicy",
        env=env,
        learning_rate = args['learning_rate'],
        n_steps = args['num_steps'],
        gamma= args['gamma'],
        verbose=1,
        device=args['device'],
        tensorboard_log=os.path.join(save_path, 'logs'),
    )
    
    model.learn(
        total_timesteps=args['timestep'],
        eval_freq=20,
        log_interval=20,
        eval_log_path=os.path.join(save_path, 'evals')
    )
    model.save(f'{save_path}/model_state_dict.pt')
    
    obs = env.reset()
    vals = []
    for i in range(80):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        vals.append(obs + [reward]) # adds reward to obs list [a,b] + [c] = [a,b,c]
        if done:
            break
    env.close()
    make_plots(vals, args, save_path)

    
    

In [68]:
args = {
    'name' : "test_runs", #the path in which the run will be saved
    "output_path": "outputs",
    "stdout": False,
    'action_space' : 36,
    "seed": 158,
    'reward_mode' : "temp_emit_diff_reward",
    'forcing' : True,
    'multigas' : True,
    'scenario' : "ssp245",
    'algorithm' : "A2C",
    'learning_rate' : 2.1e-05,
    'num_steps' : 100, 
    'gamma' : 0.9, 
    'device' : 'cpu',
    'timestep' : 200,
    "n_steps" : 5,
    
}

In [55]:
# Make output directories
def make_outdirs(args):
    saved_path = 
    dirs = ['plots', 'logs', 'saved_models']
    for direc in dirs:
        path = os.path.join(save_path, direc)
        if not os.path.exists(path):
            os.makedirs(path)

In [71]:
# Set random seeds for reproducibility
random.seed(args['seed'])
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])

# Setup save path and logging
save_path = os.path.join(args['output_path'], args['name']) # outputs/test_runs/
make_outdirs(save_path)

start_time = time.time()

if not args['stdout']:
    #sys.stdout = open(os.path.join(save_path, 'stdout.txt'), 'w')
    file_write = open(os.path.join(save_path, 'stdout.txt'), 'w')
    

with open(os.path.join(save_path, 'config.txt'), 'w') as file:
    json.dump(args, file, indent=2)


run_model(args, save_path)

# log total runtime and close logging file


if not args['stdout']:
    file_write.write(f'\nTOTAL RUNTIME: {int((time.time() - start_time)/60.)} minutes {int((time.time() - start_time) %60)} seconds')
    file_write.close()



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/var/folders/lc/mjy2_vc10zd7k8yny4sjghbr0000gn/T/ipykernel_82135/3206657176.py:26: DeprecationWarning: Parameters `eval_env` and `eval_freq` are deprecated and will be removed in the future. Please use `EvalCallback` or a custom Callback instead.
  model.learn(


ValueError: could not broadcast input array from shape (4,) into shape (16,)

# Figuring out os.path stuff

In [ ]:
os.path.join()

# Argparse

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument("--name", type=str, default='test', required=False)
parser.add_argument("--action_space", type=int, default=2, required=False)
parser.add_argument("--reward_mode", type=str, default='carbon_cost_GDP', required=False)
parser.add_argument("--forcing", action='store_true', required=False)
parser.add_argument("--output_path", type=str, default='outputs', required=False)
parser.add_argument("--stdout", action='store_true', required=False)
parser.add_argument("--seed", type=int, default=random.randint(1,1000), required=False)
parser.add_argument("--device", type=str, default='cpu', required=False)
parser.add_argument("--lr", type=float, default=2.1e-5, required=False)
parser.add_argument("--n_steps", type=int, default=5, required=False)
parser.add_argument("--gamma", type=float, default=0.99, required=False)
parser.add_argument("--timesteps", type=int, default=10000, required=False)
parser.add_argument("--algorithm", type=str, default='a2c', required=False)
parser.add_argument("--multigas", action='store_true', required=False)
parser.add_argument("--scenario", type=str, default='ssp245', required=False)
args = parser.parse_args("")

In [ ]:
# Main training and eval loop
def main(args, save_path):
    # Create the environment
    env = Simulator(
        action_space = args.action_space,
        reward_mode = args.reward_mode,
        forcing = args.forcing,
        multigas = args.multigas,
        scenario = args.scenario
    )
    
    # Train the algorithm
    if args.algorithm == 'a2c':
        model_builder = A2C
    elif args.algorithm == 'ppo':
        model_builder = PPO
    elif args.algorithm == 'ddpg':
        model_builder = DDPG
    
    model = model_builder(
        policy="MlpPolicy",
        env=env,
        learning_rate = args.lr,
        n_steps = args.n_steps,
        gamma=args.gamma,
        verbose=1,
        device=args.device,
        tensorboard_log=os.path.join(save_path, 'logs'),
    )
    model.learn(
        total_timesteps=args.timesteps,
        eval_freq=20,
        log_interval=20,
        eval_log_path=os.path.join(save_path, 'evals')
    )
    model.save(f'{save_path}/model_state_dict.pt')
    
    obs = env.reset()
    vals = []
    for i in range(80):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        vals.append(obs + [reward])
        if done:
            break
    env.close()
    make_plots(vals, args, save_path)

    

In [21]:
parser = argparse.ArgumentParser()
parser.add_argument("--name", type=str, default='test', required=False)
parser.add_argument("--action_space", type=int, default=2, required=False)
parser.add_argument("--reward_mode", type=str, default='carbon_cost_GDP', required=False)
parser.add_argument("--forcing", action='store_true', required=False)
parser.add_argument("--output_path", type=str, default='outputs', required=False)
parser.add_argument("--stdout", action='store_true', required=False)
parser.add_argument("--seed", type=int, default=random.randint(1,1000), required=False)
parser.add_argument("--device", type=str, default='cpu', required=False)
parser.add_argument("--lr", type=float, default=2.1e-5, required=False)
parser.add_argument("--n_steps", type=int, default=5, required=False)
parser.add_argument("--gamma", type=float, default=0.99, required=False)
parser.add_argument("--timesteps", type=int, default=10000, required=False)
parser.add_argument("--algorithm", type=str, default='a2c', required=False)
parser.add_argument("--multigas", action='store_true', required=False)
parser.add_argument("--scenario", type=str, default='ssp245', required=False)
args = parser.parse_args("")